<a href="https://colab.research.google.com/github/labibahanastu/MachineLearning_Assignment/blob/main/MnistSimpleCNN_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Final Exam Paper 1 
Labibah Anastuzahra P (1103180059)


#EMA

In [1]:
class EMA:
    def __init__(self, model, decay):
        self.decay = decay
        self.shadow = {}
        self.original = {}

        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def __call__(self, model, num_updates):
        decay = min(self.decay, (1.0 + num_updates) / (10.0 + num_updates))
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - decay) * param.data + decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def assign(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.original[name] = param.data.clone()
                param.data = self.shadow[name]

    def resume(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                param.data = self.original[name]



#Datasets

In [2]:
import torch
import numpy as np
from PIL import Image
from torchvision import transforms

class MnistDataset(torch.utils.data.Dataset):
    def __init__(self, training=True, transform=None):
        if training==True:
            f = open('train-images-idx3-ubyte', 'rb')
            xs = np.array(np.frombuffer(f.read(), np.uint8, offset=16))
            f.close()
            f = open('train-labels-idx1-ubyte', 'rb')
            ys = np.array(np.frombuffer(f.read(), np.uint8, offset=8))
            f.close()
        else:
            f = open('t10k-images-idx3-ubyte', 'rb')
            xs = np.array(np.frombuffer(f.read(), np.uint8, offset=16))
            f.close()
            f = open('t10k-labels-idx1-ubyte', 'rb')
            ys = np.array(np.frombuffer(f.read(), np.uint8, offset=8))
            f.close()
        xs = np.reshape(xs, (-1, 28, 28, 1)).astype(np.float32)
        ys = ys.astype(np.int)
        self.x_data = xs
        self.y_data = ys
        self.transform = transform

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = Image.fromarray(self.x_data[idx].reshape(28, 28))
        y = torch.tensor(np.array(self.y_data[idx]))
        if self.transform:
            x = self.transform(x)
        x = transforms.ToTensor()(np.array(x)/255)
        return x, y



#Transforms

In [3]:
import random
import torchvision.transforms.functional as F

class RandomRotation(object):
    def __init__(self, degrees, seed=1):
        self.degrees = (-degrees, degrees)
        random.seed(seed)
    
    @staticmethod
    def get_params(degrees):
        angle = random.uniform(degrees[0], degrees[1])
        return angle

    def __call__(self, img):
        angle = self.get_params(self.degrees)
        return F.rotate(img, angle, False, False, None, None)


#Models

###ModelM3

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ModelM3(nn.Module):
    def __init__(self):
        super(ModelM3, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, bias=False)       # output becomes 26x26
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 48, 3, bias=False)      # output becomes 24x24
        self.conv2_bn = nn.BatchNorm2d(48)
        self.conv3 = nn.Conv2d(48, 64, 3, bias=False)      # output becomes 22x22
        self.conv3_bn = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 80, 3, bias=False)      # output becomes 20x20
        self.conv4_bn = nn.BatchNorm2d(80)
        self.conv5 = nn.Conv2d(80, 96, 3, bias=False)      # output becomes 18x18
        self.conv5_bn = nn.BatchNorm2d(96)
        self.conv6 = nn.Conv2d(96, 112, 3, bias=False)     # output becomes 16x16
        self.conv6_bn = nn.BatchNorm2d(112)
        self.conv7 = nn.Conv2d(112, 128, 3, bias=False)    # output becomes 14x14
        self.conv7_bn = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(128, 144, 3, bias=False)    # output becomes 12x12
        self.conv8_bn = nn.BatchNorm2d(144)
        self.conv9 = nn.Conv2d(144, 160, 3, bias=False)    # output becomes 10x10
        self.conv9_bn = nn.BatchNorm2d(160)
        self.conv10 = nn.Conv2d(160, 176, 3, bias=False)   # output becomes 8x8
        self.conv10_bn = nn.BatchNorm2d(176)
        self.fc1 = nn.Linear(11264, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        conv5 = F.relu(self.conv5_bn(self.conv5(conv4)))
        conv6 = F.relu(self.conv6_bn(self.conv6(conv5)))
        conv7 = F.relu(self.conv7_bn(self.conv7(conv6)))
        conv8 = F.relu(self.conv8_bn(self.conv8(conv7)))
        conv9 = F.relu(self.conv9_bn(self.conv9(conv8)))
        conv10 = F.relu(self.conv10_bn(self.conv10(conv9)))
        flat1 = torch.flatten(conv10.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat1))
        return logits
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)

        #Menghasilkan output yang berbeda sesuai dengan parameter


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ModelM5(nn.Module):
    def __init__(self):
        super(ModelM5, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, bias=False)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 5, bias=False)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 96, 5, bias=False)
        self.conv3_bn = nn.BatchNorm2d(96)
        self.conv4 = nn.Conv2d(96, 128, 5, bias=False)
        self.conv4_bn = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 160, 5, bias=False)
        self.conv5_bn = nn.BatchNorm2d(160)
        self.fc1 = nn.Linear(10240, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        conv5 = F.relu(self.conv5_bn(self.conv5(conv4)))
        flat5 = torch.flatten(conv5.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat5))
        return logits
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)

        #Menghasilkan output yang berbeda sesuai dengan parameter


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ModelM7(nn.Module):
    def __init__(self):
        super(ModelM7, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, 7, bias=False)    # output becomes 22x22
        self.conv1_bn = nn.BatchNorm2d(48)
        self.conv2 = nn.Conv2d(48, 96, 7, bias=False)   # output becomes 16x16
        self.conv2_bn = nn.BatchNorm2d(96)
        self.conv3 = nn.Conv2d(96, 144, 7, bias=False)  # output becomes 10x10
        self.conv3_bn = nn.BatchNorm2d(144)
        self.conv4 = nn.Conv2d(144, 192, 7, bias=False) # output becomes 4x4
        self.conv4_bn = nn.BatchNorm2d(192)
        self.fc1 = nn.Linear(3072, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        flat1 = torch.flatten(conv4.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat1))
        return logits
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)

        #Menghasilkan output yang berbeda sesuai dengan parameter


Train

In [7]:
# imports -------------------------------------------------------------------------#
# mengimport semua modul yang digunakan
import sys #modul yang berkaitan dengan interpreter python antara lain system
import os #digunakan untuk dapat berinteraksi langsung dengan operasi sistem
import argparse #digunakan untuk pendefinisian sebuah argumen dengan tipe data yang berbeda
import numpy as np #menyimpan sebuah data dalam bentuk array dimana terdapat variable yang memiliki tipe data sama
import math #modul yang memperluas daftar fungsi matematika
import torch #pustaka tensor yang dioptimalkan dalam menggunakan GPU dan CPU
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms #digunakan untuk pengolahan citra
from torchsummary import summary
from PIL import Image #library yang digunakan untuk memanipulasi file gambar


def run(p_seed=0, p_epochs=150, p_kernel_size=5, p_logdir="temp"):
    # random number generator seed ------------------------------------------------#
    SEED = p_seed
    #akan mengaktfikan algoritma CuDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    np.random.seed(SEED)

    # kernel size of model --------------------------------------------------------#
    KERNEL_SIZE = p_kernel_size

    # number of epochs ------------------------------------------------------------#
    NUM_EPOCHS = p_epochs

    # file names ------------------------------------------------------------------#
    # data terdapat di folder "content/logs"
    if not os.path.exists("content/logs/%s"%p_logdir):
        os.makedirs("content/logs/%s"%p_logdir)
    OUTPUT_FILE = str("content/logs/%s/log%03d.out"%(p_logdir,SEED))
    MODEL_FILE = str("content/logs/%s/model%03d.pth"%(p_logdir,SEED))

    # enable GPU usage ------------------------------------------------------------#
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda == False:
      #jika GPU tidak dapat digunakan maka CPU sebagai gantinya 
        print("WARNING: CPU will be used for training.")
        exit(0)

    # data augmentation methods ---------------------------------------------------#
    transform = transforms.Compose([
        RandomRotation(20, seed=SEED),
        transforms.RandomAffine(0, translate=(0.2, 0.2)),
        ])

    # data loader -----------------------------------------------------------------#
    # data akan dimuat dengan directory yang berbeda
    train_dataset = MnistDataset(training=True, transform=transform)
    test_dataset = MnistDataset(training=False, transform=None)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=120, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

    # model selection -------------------------------------------------------------#
    # digunakan untuk dapat memilih kernel sesuai dengan modelnya
    if(KERNEL_SIZE == 3):
        model = ModelM3().to(device) #kernel 3x3
    elif(KERNEL_SIZE == 5):
        model = ModelM5().to(device) #kenel 5x5
    elif(KERNEL_SIZE == 7):
        model = ModelM7().to(device) #kernel 7x7

    summary(model, (1, 28, 28))

    # hyperparameter selection ----------------------------------------------------#
    # data akan dimuat setelah modul berhasil diinstal
    ema = EMA(model, decay=0.999)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

    # delete result file ----------------------------------------------------------#
    # hasil file yang terhapus akan disimpan dengan name 'w'
    f = open(OUTPUT_FILE, 'w')
    f.close()

    # global variables ------------------------------------------------------------#
    # Calculate Accuracy
    g_step = 0
    max_correct = 0

    # training and evaluation loop ------------------------------------------------#
    for epoch in range(NUM_EPOCHS):
        #--------------------------------------------------------------------------#
        # train process                                                            #
        #--------------------------------------------------------------------------#
        #
        model.train()
        train_loss = 0
        train_corr = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            train_pred = output.argmax(dim=1, keepdim=True)
            train_corr += train_pred.eq(target.view_as(train_pred)).sum().item()
            train_loss += F.nll_loss(output, target, reduction='sum').item()
            loss.backward()
            optimizer.step()
            g_step += 1
            ema(model, g_step)
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{:05d}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
        train_loss /= len(train_loader.dataset)
        train_accuracy = 100 * train_corr / len(train_loader.dataset)

        #--------------------------------------------------------------------------#
        # test process                                                             #
        #--------------------------------------------------------------------------#
        model.eval()
        ema.assign(model)
        test_loss = 0
        correct = 0
        total_pred = np.zeros(0)
        total_target = np.zeros(0)
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item()
                pred = output.argmax(dim=1, keepdim=True)
                total_pred = np.append(total_pred, pred.cpu().numpy())
                total_target = np.append(total_target, target.cpu().numpy())
                correct += pred.eq(target.view_as(pred)).sum().item()
            if(max_correct < correct):
                torch.save(model.state_dict(), MODEL_FILE)
                max_correct = correct
                print("Best accuracy! correct images: %5d"%correct)
        ema.resume(model)

        #--------------------------------------------------------------------------#
        # output                                                                   #
        #--------------------------------------------------------------------------#
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100 * correct / len(test_loader.dataset)
        best_test_accuracy = 100 * max_correct / len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%) (best: {:.2f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), test_accuracy, best_test_accuracy))

        f = open(OUTPUT_FILE, 'a')
        f.write(" %3d %12.6f %9.3f %12.6f %9.3f %9.3f\n"%(epoch, train_loss, train_accuracy, test_loss, test_accuracy, best_test_accuracy))
        f.close()

        #output yang dihasilkan setelah train process dan test process akan menampilkan epoch, train_loss, train_accuracy, test_loss, test_accuracy, best_test_accuracy 

        #--------------------------------------------------------------------------#
        # update learning rate scheduler                                           #
        #--------------------------------------------------------------------------#
        lr_scheduler.step()

        if __name__ == "__main__":
          #pendefisinian sebuah argumen processing dengan tipe data integer
            p = argparse.ArgumentParser()
            p.add_argument("--seed", default=0, type=int) #argumen --seed memiliki nilai default 0 yang akan dikonversikan kedalam integer
            p.add_argument("--trials", default=15, type=int) #argumen --trials memiliki nilai default 15 yang akan dikonversikan kedalam integer
            p.add_argument("--epochs", default=150, type=int) #argumen --epoch memiliki nilai default 150 yang akan dikonversikan kedalam integer    
            p.add_argument("--kernel_size", default=5, type=int) #argumen --kernel size memiliki nilai default 5 yang akan dikonversikan kedalam integer   
            p.add_argument("--gpu", default=0, type=int) #argumen --gpu memiliki nilai default 0 yang akan dikonversikan kedalam integer
            p.add_argument("--logdir", default="temp") #argumen --logdir memiliki nilai default "temp" yang akan dikonversikan kedalam integer
            args = p.parse_args()
            os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
            os.environ["CUDA_VISIBLE_DEVICES"]=str(args.gpu)
            for i in range(args.trials):
                run(p_seed = args.seed + i,
                    p_epochs = args.epochs,
                    p_kernel_size = args.kernel_size,
                    p_logdir = args.logdir)


Test

In [9]:
# imports ---------------------------------------------------------------------#
# mengimport semua modul yang digunakan
import sys #modul yang berkaitan dengan interpreter python antara lain system
import os #digunakan untuk dapat berinteraksi langsung dengan operasi sistem
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import argparse #digunakan untuk pendefinisian sebuah argumen dengan tipe data yang berbeda
import numpy as np #menyimpan sebuah data dalam bentuk array dimana terdapat variable yang memiliki tipe data sama
import math #modul yang memperluas daftar fungsi matematika
import torch #pustaka tensor yang dioptimalkan dalam menggunakan GPU dan CPU
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms #digunakan untuk pengolahan citra
from PIL import Image #library yang digunakan untuk memanipulasi file gambar
#from ema import EMA 
#from datasets import MnistDataset #untuk memudahkan proses pembacaan data
#from transforms import RandomRotation #digunakan untuk pengolahan awal dataset
#model yang di import dari folder models
#from modelM3 import ModelM3
#from modelM5 import ModelM5
#from modelM7 import ModelM7



def run(p_seed=0, p_kernel_size=5, p_logdir="temp"):

    # enable GPU usage ------------------------------------------------------------#
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    #jika GPU tidak dapat digunakan maka CPU sebagai gantinya 
    if use_cuda == False:
        print("WARNING: CPU will be used for training.")
        exit(0)

    # data loader -----------------------------------------------------------------#
    # memuat data yang sudah diimport
    test_dataset = MnistDataset(training=False, transform=None)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

    # model selection -------------------------------------------------------------#
    # digunakan untuk dapat memilih kernel sesuai dengan modelnya
    if(p_kernel_size == 3): #kernel 3x3
        model1 = ModelM3().to(device)
    elif(p_kernel_size == 5): #kernel 5x5
        model1 = ModelM5().to(device)
    elif(p_kernel_size == 7): #kernel 7x7
        model1 = ModelM7().to(device)

    model1.load_state_dict(torch.load("logs/%s/model%03d.pth"%(p_logdir,p_seed))) #memuat data kernel model

    model1.eval()
    test_loss = 0
    correct = 0
    wrong_images = [] #digunakan untuk menyimpan gambar apabila terdapat kesalahan ketika klasifikasi oleh model
    with torch.no_grad(): #menggunakan pytoch hanya untuk operasi matematika tanpa propagasi balik
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model1(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            wrong_images.extend(np.nonzero(~pred.eq(target.view_as(pred)).cpu().numpy())[0]+(100*batch_idx))

    np.savetxt("logs/%s/wrong%03d.txt"%(p_logdir,p_seed), wrong_images, fmt="%d")
    #print(len(wrong_images), wrong_images)

    
if __name__ == "__main__":
  #pendefisinian sebuah argumen processing dengan tipe data integer
    p = argparse.ArgumentParser()
    p.add_argument("--logdir", default="modelM5") #argumen --logdir yang memiliki nilai default modelM5
    p.add_argument("--seed", default=0, type=int) #argumen --seed memiliki nilai default 0 yang akan dikonversikan kedalam integer
    p.add_argument("--trials", default=30, type=int) #argumen --trials memiliki nilai default 30 yang akan dikonversikan kedalam integer
    p.add_argument("--kernel_size", default=5, type=int) #argumen --kernel_size memiliki nilai default 5 yang akan dikonversikan kedalam integer
    p.add_argument("--epochs", default=150, type=int)  
    args = p.parse_args() 
    for i in range(args.trials):
        run(p_seed = args.seed + i,
            p_kernel_size = args.kernel_size,
            p_logdir = args.logdir)
        #terdapat error saat run args = p.parse_args() karena file-file yang seharusnya masuk ke dalam directory logs tidak dapat ter-load 

usage: ipykernel_launcher.py [-h] [--logdir LOGDIR] [--seed SEED]
                             [--trials TRIALS] [--kernel_size KERNEL_SIZE]
                             [--epochs EPOCHS]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-bf7bb264-e9e5-4c25-8981-07f19b09054d.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Homo Ensemble

In [10]:
#Import libraries
import numpy as np # linear algebra 
import argparse 

cnt = 1
best = 10000
curr = 10000

#pendefisinian sebuah argumen processing dengan tipe data integer
p = argparse.ArgumentParser()
p.add_argument("--kernel_size", default=5, type=int) #argumen --kernel_size memiliki nilai default 5 yang akan dikonversikan kedalam integer
args = p.parse_args()
KERNEL_SIZE = args.kernel_size

for i in range(10):
    for j in range(i+1,10):
        for k in range(j+1,10):
            w1 = np.loadtxt("logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, i)).astype(np.int) #load file in logs directory
            w2 = np.loadtxt("logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, j)).astype(np.int)
            w3 = np.loadtxt("logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, k)).astype(np.int)

            board = np.zeros((10000))
            board[w1] += 1
            board[w2] += 1
            board[w3] += 1
            board = board // 2
            curr = np.sum(board)
            if curr < best:
                best = curr
            print("%4d %4d %4d %4d %4d %4d"%(cnt, len(w1), len(w2), len(w3), curr, best))
            cnt += 1


usage: ipykernel_launcher.py [-h] [--kernel_size KERNEL_SIZE]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-bf7bb264-e9e5-4c25-8981-07f19b09054d.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
